# Load libraries

In [1]:
import sys
#!{sys.executable} -m 
import glob
from functools import reduce
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, Ridge, ElasticNet, SGDRegressor, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, LeaveOneOut, train_test_split, cross_val_score, \
                                    cross_val_predict, StratifiedKFold, KFold, GroupKFold, LeavePGroupsOut
from scipy.stats import pearsonr
#from adaptivesplit import ml
from sklearn.feature_selection import SelectKBest, f_regression, VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score, explained_variance_score


#from statsmodels.graphics.gofplots import qqplot
#!{sys.executable} -m pip install mlxtend
#from mlxtend.evaluate import permutation_test
#from PAINTeR import plot
from neurocombat_sklearn import CombatModel
#from neuroCombat import neuroCombat
from scipy.stats import normaltest
from mlxtend.evaluate import permutation_test

# Models

In [2]:
class models_ml:
    def __init__(self, model_name, feature_size):
        self.model_name = model_name
        self.feature_size = feature_size
    
    def get_model(self):
        if self.model_name in {'modelLasso'}:
            return self.modelLasso()
        if self.model_name in {'modelRidge'}:
            return self.modelRidge()
        if self.model_name in {'elasticNet'}:
            return self.elasticNet()
        if self.model_name in {'SGDRegressor'}:
            return self.SGDRegressor()
        if self.model_name in {'modelLasso_nf'}:
            return self.modelLasso_nf()
        if self.model_name in {'modelLasso_scaler'}:
            return self.modelLasso_scaler()
        if self.model_name in {'modelLasso_tol'}:
            return self.modelLasso_tol()
        
    def modelLasso(self):
        model = Pipeline([('fsel',SelectKBest(f_regression)),             
                          ('model_Lasso', Lasso(max_iter=100000))
                         ])
        p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                  'model_Lasso__alpha':[0.001, 0.01, 0.1, 1, 10, 100, 1000]} 
        return model, p_grid
    
    def modelLasso_tol(self):
        model = Pipeline([('fsel',SelectKBest(f_regression)),             
                          ('model_Lasso', Lasso(max_iter=100000))
                         ])
        p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                  'model_Lasso__alpha':[0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                 'model_Lasso__tol':[0.0001, 0.0001, 0.001, 0.01]} 
        return model, p_grid
    
    def modelLasso_scaler(self):
        model = Pipeline([('scaler',preprocessing.StandardScaler()),
                          ('fsel',SelectKBest(f_regression)),             
                          ('model_Lasso', Lasso(max_iter=100000))
                         ])
        p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                  'model_Lasso__alpha':[1e-3, 1e-2, 
                                        0.1, 1e+2, 1e+3, 10000]} 
        return model, p_grid
    
    def modelLasso_nf(self):
        model = Pipeline([         
                          ('model_Lasso', Lasso(normalize=False,max_iter=100000))
                         ])
        p_grid = {
                  'model_Lasso__alpha':[1e-9, 1e-8, 1e-7, 1e-6,1e-5,1e-4,1e-3, 1e-2, 
                                        0.1, 1e+2, 1e+3, 10000,100000,1000000, 10000000, 100000000, 1000000000]} 
        return model, p_grid
    
    def modelRidge(self):
        model = Pipeline([('fsel',SelectKBest(f_regression)),
                          ('model_Ridge', Ridge(max_iter=100000))])
        if self.feature_size < 300:
            p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                      'model_Ridge__alpha':[1e-3, 1e-2, 
                                        0.1, 1e+2, 1e+3, 10000]}
        else:
            p_grid = {'fsel__k':np.arange(5,200,5),
                      'model_Ridge__alpha':[1e-3, 1e-2, 
                                        0.1, 1e+2, 1e+3, 10000]}
            
        return model, p_grid

    def elasticNet(self):
        model = Pipeline([('fsel',SelectKBest(f_regression)),
                          ('model_en', ElasticNet(max_iter=100000))])
        if self.feature_size < 300:
            p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                      'model_en__alpha': [[1e-3, 1e-2, 
                                        0.1, 1e+2, 1e+3, 10000]],
                      'model_en__l1_ratio': [0.0001, .25, .5, .75, 0.9999]}
        return model, p_grid 
   
    def SGDRegressor(self):
        model = Pipeline([('fsel',SelectKBest(f_regression)), # remove features
                          ('model_sgd', SGDRegressor(max_iter=100000))])
        if self.feature_size < 300:
            p_grid = {'fsel__k':np.arange(5,self.feature_size,5),
                     'model_sgd__alpha': [.001, .005, .01, .05, .1, .5, 1, 5],}
        return model, p_grid


In [3]:
data = pd.read_csv('../data/data_unbias/hcp1200_behavioral_data.csv', sep = ',',index_col=0)

<ipython-input-3-2ed22bff8901>:1: DtypeWarning: Columns (82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/data_unbias/hcp1200_behavioral_data.csv', sep = ',',index_col=0)


# headers 

In [4]:
fs_rois = [
       'FS_L_Parahippocampal_Thck', 'FS_R_Parahippocampal_Thck',
       'FS_L_Rostralanteriorcingulate_Thck','FS_R_Rostralanteriorcingulate_Thck',
       'FS_L_Temporalpole_Thck', 'FS_R_Temporalpole_Thck',
       'FS_L_Parahippocampal_Area', 'FS_R_Parahippocampal_Area',
       'FS_L_Rostralanteriorcingulate_Area','FS_R_Rostralanteriorcingulate_Area',
       'FS_L_Temporalpole_Area', 'FS_R_Temporalpole_Area',
        ]

fs_thck_lh = [
       'FS_L_Bankssts_Thck', 'FS_L_Caudalanteriorcingulate_Thck',
       'FS_L_Caudalmiddlefrontal_Thck', 'FS_L_Cuneus_Thck',
       'FS_L_Entorhinal_Thck', 'FS_L_Fusiform_Thck',
       'FS_L_Inferiorparietal_Thck', 'FS_L_Inferiortemporal_Thck',
       'FS_L_Isthmuscingulate_Thck', 'FS_L_Lateraloccipital_Thck',
       'FS_L_Lateralorbitofrontal_Thck', 'FS_L_Lingual_Thck',
       'FS_L_Medialorbitofrontal_Thck', 'FS_L_Middletemporal_Thck',
       'FS_L_Parahippocampal_Thck', 'FS_L_Paracentral_Thck',
       'FS_L_Parsopercularis_Thck', 'FS_L_Parsorbitalis_Thck',
       'FS_L_Parstriangularis_Thck', 'FS_L_Pericalcarine_Thck',
       'FS_L_Postcentral_Thck', 'FS_L_Posteriorcingulate_Thck',
       'FS_L_Precentral_Thck', 'FS_L_Precuneus_Thck',
       'FS_L_Rostralanteriorcingulate_Thck',
       'FS_L_Rostralmiddlefrontal_Thck', 'FS_L_Superiorfrontal_Thck',
       'FS_L_Superiorparietal_Thck', 'FS_L_Superiortemporal_Thck',
       'FS_L_Supramarginal_Thck', 'FS_L_Frontalpole_Thck',
       'FS_L_Temporalpole_Thck', 'FS_L_Transversetemporal_Thck',
       'FS_L_Insula_Thck'
        ]

fs_thck_rh = [
       'FS_R_Bankssts_Thck',
       'FS_R_Caudalanteriorcingulate_Thck',
       'FS_R_Caudalmiddlefrontal_Thck', 'FS_R_Cuneus_Thck',
       'FS_R_Entorhinal_Thck', 'FS_R_Fusiform_Thck',
       'FS_R_Inferiorparietal_Thck', 'FS_R_Inferiortemporal_Thck',
       'FS_R_Isthmuscingulate_Thck', 'FS_R_Lateraloccipital_Thck',
       'FS_R_Lateralorbitofrontal_Thck', 'FS_R_Lingual_Thck',
       'FS_R_Medialorbitofrontal_Thck', 'FS_R_Middletemporal_Thck',
       'FS_R_Parahippocampal_Thck', 'FS_R_Paracentral_Thck',
       'FS_R_Parsopercularis_Thck', 'FS_R_Parsorbitalis_Thck',
       'FS_R_Parstriangularis_Thck', 'FS_R_Pericalcarine_Thck',
       'FS_R_Postcentral_Thck', 'FS_R_Posteriorcingulate_Thck',
       'FS_R_Precentral_Thck', 'FS_R_Precuneus_Thck',
       'FS_R_Rostralanteriorcingulate_Thck',
       'FS_R_Rostralmiddlefrontal_Thck', 'FS_R_Superiorfrontal_Thck',
       'FS_R_Superiorparietal_Thck', 'FS_R_Superiortemporal_Thck',
       'FS_R_Supramarginal_Thck', 'FS_R_Frontalpole_Thck',
       'FS_R_Temporalpole_Thck', 'FS_R_Transversetemporal_Thck',
       'FS_R_Insula_Thck'
        ]

icv = 'FS_IntraCranial_Vol'

behavior = [
    'PainIntens_RawScore','PainInterf_Tscore'

]

In [5]:
fs_thick_lh = data[fs_thck_lh]
fs_thick_rh = data[fs_thck_rh]
fs_rois_hyp = data[fs_rois]
eTIV = data[icv]
y1 = pd.DataFrame(data['PainIntens_RawScore'])
y2 = pd.DataFrame(data['PainInterf_Tscore'])
y1.head()

,PainIntens_RawScore
Subject,
100307,0.0
103818,0.0
111312,0.0
114924,2.0
117122,1.0


# X_norm

In [6]:
lh_meanthickness = fs_thick_lh.T.mean()
rh_meanthickness = fs_thick_rh.T.mean()

fs_l_phg_thick = fs_rois_hyp['FS_L_Parahippocampal_Thck'].div(lh_meanthickness, axis = 0)
fs_r_phg_thick = fs_rois_hyp['FS_R_Parahippocampal_Thck'].div(rh_meanthickness, axis = 0)

fs_l_racc_thick = fs_rois_hyp['FS_L_Rostralanteriorcingulate_Thck'].div(lh_meanthickness, axis = 0)
fs_r_racc_thick = fs_rois_hyp['FS_R_Rostralanteriorcingulate_Thck'].div(rh_meanthickness, axis = 0)

fs_l_tp_thick = fs_rois_hyp['FS_L_Temporalpole_Thck'].div(lh_meanthickness, axis = 0)
fs_r_tp_thick = fs_rois_hyp['FS_R_Temporalpole_Thck'].div(rh_meanthickness, axis = 0)

fs_l_phg_area = fs_rois_hyp['FS_L_Parahippocampal_Area'].div(eTIV, axis = 0)
fs_r_phg_area = fs_rois_hyp['FS_L_Parahippocampal_Area'].div(eTIV, axis = 0)

fs_l_racc_area = fs_rois_hyp['FS_L_Rostralanteriorcingulate_Area'].div(eTIV, axis = 0)
fs_r_racc_area = fs_rois_hyp['FS_R_Rostralanteriorcingulate_Area'].div(eTIV, axis = 0)

fs_l_tp_area = fs_rois_hyp['FS_L_Temporalpole_Area'].div(eTIV, axis = 0)
fs_r_tp_area = fs_rois_hyp['FS_R_Temporalpole_Area'].div(eTIV, axis = 0)

X_feature = pd.concat([fs_l_phg_thick, fs_r_phg_thick, fs_l_racc_thick,fs_r_racc_thick, 
                      fs_l_tp_thick, fs_r_tp_thick,fs_l_phg_area, fs_r_phg_area, fs_l_racc_area,fs_r_racc_area, 
                      fs_l_tp_area, fs_r_tp_area, ],axis=1)
X_feature.columns = ['lphg_thick', 'rphg_thick', 'lracc_thick', 'rracc_thick', 'ltp_thick','rtp_thick',
                    'lphg_area', 'rphg_area', 'lracc_area', 'rracc_area', 'ltp_area','rtp_area']
X_feature.head()

,lphg_thick,rphg_thick,lracc_thick,rracc_thick,ltp_thick,rtp_thick,lphg_area,rphg_area,lracc_area,rracc_area,ltp_area,rtp_area
Subject,,,,,,,,,,,,
100307,0.882130,0.990907,1.096207,1.060629,1.295224,1.417694,0.000502,0.000502,0.000489,0.000334,0.000303,0.000272
103818,1.158432,1.144297,1.147323,1.114584,1.328187,1.374746,0.000400,0.000400,0.000497,0.000425,0.000351,0.000310
111312,0.982679,0.928807,1.185590,1.149682,1.305315,1.406048,0.000458,0.000458,0.000493,0.000443,0.000331,0.000266
114924,0.984247,1.035941,1.236187,1.230385,1.449310,1.451775,0.000410,0.000410,0.000488,0.000493,0.000306,0.000274
117122,1.136194,1.155880,1.070802,1.058347,1.388475,1.497246,0.000466,0.000466,0.000496,0.000413,0.000357,0.000305


# covariates

In [7]:
covariates = data[['Age','Gender','FS_IntraCranial_Vol','Batch']]
covariates.head()
cov = np.array(covariates)
covariates

,Age,Gender,FS_IntraCranial_Vol,Batch
Subject,,,,
100307,26-30,0,1512540.231,1
103818,31-35,0,1456673.807,1
111312,31-35,0,1404835.234,1
114924,26-30,1,1576894.860,1
117122,26-30,0,1411284.193,1
...,...,...,...,...
888678,26-30,1,1672947.090,13
911849,31-35,0,1593657.781,13
973770,22-25,1,1523394.069,13


# regress TIV

In [8]:
res = []
lr = LinearRegression()
for i in np.arange(0,np.size(X_feature,1),1):
    lr.fit(pd.DataFrame(cov[:,2]),X_feature.iloc[:,i])
    lr_pred = lr.predict(pd.DataFrame(cov[:,2]))
    resid = X_feature.iloc[:,i] - np.array(lr_pred)
    res.append(resid)
df_X = pd.DataFrame(res)
X_norm = df_X.transpose()

In [9]:
age = []
for s in covariates['Age']:
    split = s.split(sep='-')
    age.append(np.mean((float(split[0]), float(split[1]))))

covariates['Age'] = age
covariates.head()

<ipython-input-9-1eed2f02441e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covariates['Age'] = age


,Age,Gender,FS_IntraCranial_Vol,Batch
Subject,,,,
100307,28.0,0,1512540.231,1
103818,33.0,0,1456673.807,1
111312,33.0,0,1404835.234,1
114924,28.0,1,1576894.860,1
117122,28.0,0,1411284.193,1


# merge all!

In [10]:
X_norm.head()

,lphg_thick,rphg_thick,lracc_thick,rracc_thick,ltp_thick,rtp_thick,lphg_area,rphg_area,lracc_area,rracc_area,ltp_area,rtp_area
Subject,,,,,,,,,,,,
100307,-0.140676,-0.008458,-0.039337,-0.056205,0.012838,0.070230,0.000028,0.000028,-0.000068,-0.000096,-0.000025,-0.000017
103818,0.132716,0.143760,0.011105,-0.003589,0.044270,0.027272,-0.000080,-0.000080,-0.000062,-0.000006,0.000016,0.000013
111312,-0.045737,-0.072818,0.048746,0.030265,0.019978,0.058565,-0.000029,-0.000029,-0.000068,0.000010,-0.000011,-0.000038
114924,-0.035207,0.037927,0.101420,0.115094,0.168686,0.104322,-0.000056,-0.000056,-0.000067,0.000066,-0.000013,-0.000006
117122,0.108114,0.154391,-0.065965,-0.060915,0.103315,0.149765,-0.000020,-0.000020,-0.000065,-0.000020,0.000016,0.000002


In [11]:
df = pd.merge(X_norm, covariates, left_index=True, right_index=True, how= 'left')
df_X_norm = df.iloc[:,0:12]
df = pd.merge(df, y1, left_index=True, right_index=True, how= 'left')
df = pd.merge(df, y2, left_index=True, right_index=True, how= 'left') 

# address batch effects

In [12]:
X_harmonized = pd.DataFrame(CombatModel().fit_transform(df_X_norm,
                                   np.array([df.Batch.values]).transpose(),
                                   np.array([df.Gender.values]).transpose(),
                                   np.array([df.Age.values]).transpose()
                                  ))
X_harmonized.columns = df_X_norm.columns
X_harmonized.index = df_X_norm.index
X_harmonized.head()

,lphg_thick,rphg_thick,lracc_thick,rracc_thick,ltp_thick,rtp_thick,lphg_area,rphg_area,lracc_area,rracc_area,ltp_area,rtp_area
Subject,,,,,,,,,,,,
100307,-0.150346,-0.006968,-0.032737,-0.059280,0.019833,0.077645,0.000032,0.000032,-0.000057,-0.000103,-0.000027,-0.000014
103818,0.144280,0.148030,0.010762,-0.004867,0.054408,0.035210,-0.000080,-0.000080,-0.000050,-0.000008,0.000016,0.000019
111312,-0.048383,-0.072623,0.043277,0.030159,0.026769,0.066204,-0.000027,-0.000027,-0.000057,0.000010,-0.000012,-0.000038
114924,-0.036742,0.040365,0.090069,0.117369,0.198709,0.111333,-0.000055,-0.000055,-0.000056,0.000069,-0.000015,-0.000004
117122,0.118256,0.158944,-0.055738,-0.064154,0.122773,0.156420,-0.000018,-0.000018,-0.000054,-0.000022,0.000016,0.000007


In [13]:
X_data = pd.merge(X_harmonized, y1.dropna(), left_index=True, right_index=True, how= 'right')
X_data = pd.merge(X_data, y2.dropna(), left_index=True, right_index=True, how= 'right')
X_data = pd.merge(X_data, covariates, left_index=True, right_index=True, how= 'left')

# run permutations

In [14]:
X_data.to_csv('hcp_check.csv')

In [15]:
rois_list = ['lphg_thick', 'rphg_thick', 'lracc_thick', 'rracc_thick', 'ltp_thick','rtp_thick',
                    'lphg_area', 'rphg_area', 'lracc_area', 'rracc_area', 'ltp_area','rtp_area']
y1.columns[0]

'PainIntens_RawScore'

In [16]:
data_final = pd.read_csv('hcp_check.csv', sep = ',',index_col=0)

In [17]:
data_final['PainIntens_RawScore'].shape

(1099,)

# painIntens_rawScore

In [21]:
data_final.head()

,lphg_thick,rphg_thick,lracc_thick,rracc_thick,ltp_thick,rtp_thick,lphg_area,rphg_area,lracc_area,rracc_area,ltp_area,rtp_area,PainIntens_RawScore,PainInterf_Tscore,Age,Gender,FS_IntraCranial_Vol,Batch
Subject,,,,,,,,,,,,,,,,,,
100307,-0.150346,-0.006968,-0.032737,-0.059280,0.019833,0.077645,0.000032,0.000032,-0.000057,-0.000103,-0.000027,-0.000014,0.0,38.6,28.0,0,1512540.231,1
103818,0.144280,0.148030,0.010762,-0.004867,0.054408,0.035210,-0.000080,-0.000080,-0.000050,-0.000008,0.000016,0.000019,0.0,38.6,33.0,0,1456673.807,1
111312,-0.048383,-0.072623,0.043277,0.030159,0.026769,0.066204,-0.000027,-0.000027,-0.000057,0.000010,-0.000012,-0.000038,0.0,38.6,33.0,0,1404835.234,1
114924,-0.036742,0.040365,0.090069,0.117369,0.198709,0.111333,-0.000055,-0.000055,-0.000056,0.000069,-0.000015,-0.000004,2.0,56.0,28.0,1,1576894.860,1
117122,0.118256,0.158944,-0.055738,-0.064154,0.122773,0.156420,-0.000018,-0.000018,-0.000054,-0.000022,0.000016,0.000007,1.0,48.7,28.0,0,1411284.193,1


In [25]:
temp_1=[]
for i in rois_list:
    print(i)
    temp = data_final.dropna(axis=0)
    p_corr = round(permutation_test(temp[i], temp['PainIntens_RawScore'],
                           func=lambda x,y: -np.corrcoef(x, y)[0,1],
                           method='approximate',
                           num_rounds=10000,
                           seed=42),2)
    corr = np.corrcoef(temp[i], temp['PainIntens_RawScore'])[0,1]
    print('non-nested r = ' + str(round(corr,2)) + ' permutation p =' + str(p_corr))
    temp_1.append([i,p_corr,corr])
pd.DataFrame(temp_1).to_csv('../output/hcp_corr_pain_intensity_raw_score.csv')

lphg_thick
non-nested r = 0.01 permutation p =0.66
rphg_thick
non-nested r = -0.0 permutation p =0.51
lracc_thick
non-nested r = 0.0 permutation p =0.58
rracc_thick
non-nested r = 0.04 permutation p =0.9
ltp_thick
non-nested r = -0.06 permutation p =0.02
rtp_thick
non-nested r = -0.06 permutation p =0.02
lphg_area
non-nested r = -0.04 permutation p =0.1
rphg_area
non-nested r = -0.04 permutation p =0.1
lracc_area
non-nested r = 0.01 permutation p =0.59
rracc_area
non-nested r = 0.02 permutation p =0.79
ltp_area
non-nested r = -0.04 permutation p =0.07
rtp_area
non-nested r = -0.03 permutation p =0.17


# pain PainInterf_Tscore

In [26]:
temp_2=[]
for i in rois_list:
    print(i)
    temp2 = data_final.dropna(axis=0)
    p_corr = round(permutation_test(temp2[i], temp2['PainInterf_Tscore'],
                           func=lambda x,y: -np.corrcoef(x, y)[0,1],
                           method='approximate',
                           num_rounds=10000,
                           seed=42),2)
    corr = np.corrcoef(temp2[i], temp2['PainInterf_Tscore'])[0,1]
    print('non-nested r = ' + str(round(corr,2)) + ' permutation p =' + str(p_corr) )
    temp_2.append([i,p_corr,corr])
    pd.DataFrame(temp_2).to_csv('../output/hcp_corr_pain_interf_tscore.csv')

lphg_thick
non-nested r = 0.0 permutation p =0.52
rphg_thick
non-nested r = 0.02 permutation p =0.72
lracc_thick
non-nested r = -0.02 permutation p =0.21
rracc_thick
non-nested r = 0.02 permutation p =0.79
ltp_thick
non-nested r = -0.01 permutation p =0.33
rtp_thick
non-nested r = -0.03 permutation p =0.2
lphg_area
non-nested r = -0.06 permutation p =0.02
rphg_area
non-nested r = -0.06 permutation p =0.02
lracc_area
non-nested r = 0.01 permutation p =0.6
rracc_area
non-nested r = 0.01 permutation p =0.64
ltp_area
non-nested r = -0.02 permutation p =0.22
rtp_area
non-nested r = -0.03 permutation p =0.15
